<a href="https://colab.research.google.com/github/HatemMoushir/Sentiment/blob/main/sent140_multilingual_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ✅ تثبيت المكتبات المطلوبة
!pip install -q datasets transformers evaluate pandas scikit-learn

import re
import pandas as pd
import numpy as np
from datasets import load_dataset, concatenate_datasets, Dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import evaluate
import os

# ✅ تحميل بيانات التغريدات العربية من UCI

!wget -q https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/m88gg52wp7-1.zip -O arabicdata1.zip
!unzip -o arabicdata1.zip

# Read the CSV with the correct separator and rename the column
df_ar = pd.read_csv("/content/Arabic Tweets Sentiment Classification 2024/Arabic_tweets_sentiment.csv", encoding='utf-8', sep='\t')
df_ar.rename(columns={'class': 'class'}, inplace=True)


# فلترة السجلات التي تحتوي فقط على Positive أو Negative
df_ar = df_ar[df_ar["class"].isin(["Positive", "Negative"])]


# 🧼 تنظيف النصوص الإنجليزية
def clean_english_text(text):
    text = re.sub(r"http\S+", "", text)     # remove URLs
    text = re.sub(r"@\w+", "", text)        # remove mentions
    text = re.sub(r"#", "", text)           # remove hashtags
    text = re.sub(r"[^\w\s]", "", text)     # remove punctuation
    text = re.sub(r"\d+", "", text)         # remove numbers
    return text.lower().strip()

# 🧼 تطبيع وتنظيف النصوص العربية
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

def clean_arabic_text(text):
    text = normalize_arabic(text)
    text = re.sub(r"http\S+", "", text)  # remove URLs
    text = re.sub(r"[^\u0600-\u06FF\s]", "", text)  # remove non-Arabic
    text = re.sub(r"\s+", " ", text)     # normalize whitespace
    return text.strip()

# 🗂️ تجهيز بيانات التغريدات العربية من UCI

# تنظيف وتطبيع
ds_ar["text"] = ds_ar["text"].fillna("").apply(clean_arabic_text)

ds_ar["label"] =ds_ar["class"].replace({"Negative": 0, "Positive": 1}).astype(int)


# اختيار 1000 عينة
ds_ar = ds_ar.sample(1000, random_state=42).reset_index(drop=True)

# تحويل إلى Dataset
ds_ar = Dataset.from_pandas(df_ar[["text", "label"]])

# 2️⃣ تحميل 1000 تغريدة إنجليزية من Sentiment140 (Stanford)
!wget -q https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip -O sentiment140.zip
!unzip -o sentiment140.zip

import pandas as pd
import os

cols = ['label', 'id', 'date', 'query', 'user', 'text']
ds_en = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1', names=cols)

# 3. تحويل التصنيفات:
# 0 → سلبي | 4 → إيجابي → نحولها لـ 1
ds_en = ds_en[ds_en['label'].isin([0, 4])]
ds_en['label'] =ds_en['label'].map({0: 0, 4: 1})

# 4. تقليل الحجم لعينة صغيرة للتجربة
ds_en = ds_en.sample(1000, random_state=42).reset_index(drop=True)

# 5. تحويل إلى Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(ds_en[['text', 'label']])

# 3️⃣ دمج اللغتين في Dataset واحد
ds = concatenate_datasets([
    ds_ar.remove_columns([col for col in ds_ar.column_names if col not in ["text", "label"]]),
    ds_en.remove_columns([col for col in ds_en.column_names if col not in ["text", "label"]])
]).shuffle(seed=44)

print("✅ إجمالي العبارات:", len(ds))

# 4️⃣ إعداد Tokenizer و Model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

# 5️⃣ تشفير النصوص
def tok(x):
    return tokenizer(x["text"], truncation=True, padding="max_length", max_length=128)

ds = ds.map(tok, batched=True)

# 6️⃣ تقسيم البيانات
split = ds.train_test_split(test_size=0.1, seed=45)
train_ds, test_ds = split["train"], split["test"]

# 7️⃣ إعداد التدريب
accuracy = evaluate.load("accuracy")

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=-1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds)
    }

training_args = TrainingArguments(
    output_dir="sentiment_ar_en_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    logging_steps=50,
    load_best_model_at_end=True,
    report_to=None
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 8️⃣ بدء التدريب
trainer.train()

# 9️⃣ حفظ النموذج
trainer.save_model("sentiment_model_final")
tokenizer.save_pretrained("sentiment_model_final")

Archive:  arabicdata1.zip
  inflating: Arabic Tweets Sentiment Classification 2024/Arabic_tweets_sentiment.csv  


/tmp/ipython-input-5-2506969151.py:58: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ar["label"] =df_ar["class"].replace({"Negative": 0, "Positive": 1}).astype(int)


Archive:  sentiment140.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


NameError: name 'df_small' is not defined

In [16]:

import pandas as pd

# تحميل الملف من المسار الصحيح
df = pd.read_csv("/content/Arabic Tweets Sentiment Classification 2024/Arabic_tweets_sentiment.csv", sep='\t')

# عرض أول 5 صفوف للتأكيد
print(df.columns)              # عرض أسماء الأعمدة
print(df.dtypes)              # نوع كل عمود
print(df.nunique())           # عدد القيم الفريدة في كل عمود
df.sample(10)                  # عرض 5 صفوف عشوائية

Index(['Unnamed: 0', 'text', 'class', 'id'], dtype='object')
Unnamed: 0     int64
text          object
class         object
id            object
dtype: object
Unnamed: 0    2286128
text          1191430
class               3
id            1436203
dtype: int64


,Unnamed: 0,text,class,id
1341750,1343933,الاردن مشكلة قناة غوردن عملالها قلبان,Neutral,"""1716485619315085478"""
235830,237321,وانا مواطنه اردنيه وهؤلاء يمثلوني ويمثلون عربي...,Positive,"""1730654567195382020"""
1558667,1561857,عمي الاردن اللي علم العرب السياسة عملنا انتخاب...,Neutral,"""1723787636249088172"""
2002497,2006783,أبوعبيدة غزةالعزة ضربةالقرن الشهيدالساجد استهد...,Negative,"""1741413646595506250"""
537225,539239,الله يرحمها ويصبرك,Neutral,"""1713598381681975622"""
1613834,1617289,اهلنا الاردن,Positive,"""1727729792735158760"""
975786,977800,‌‌‌‌يا دبر ليوسف أمره دبر أمري أصلح حالي أمري ...,Positive,"""1713192359628861798"""
886726,888740,احمي اهل غزة,Positive,"""1711175274249748603"""
591727,593741,ريتها مكسورة مشلولة الايد الي بدها تنمد على رج...,Negative,"""1714692788841857263"""
908682,910696,نفسي ارسم علم فلسطين على صلعته,Neutral,"""1711829339908505757"""


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [15]:
import pandas as pd

# تحميل الملف مع تحديد الفاصل الصحيح (tab)
df = pd.read_csv("Arabic Tweets sentiment classfication 2024/Arabic_tweets_sentiment.csv", sep='\t')

# عرض أول 5 صفوف للتأكد
df.sample(10)

FileNotFoundError: [Errno 2] No such file or directory: 'Arabic Tweets sentiment classfication 2024/Arabic_tweets_sentiment.csv'

In [10]:
!wget -q https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/m88gg52wp7-1.zip -O arabicdata1.zip
!unzip -o arabicdata1.zip

Archive:  arabicdata1.zip
  inflating: Arabic Tweets Sentiment Classification 2024/Arabic_tweets_sentiment.csv  
